# Data
harry potter book - https://www.kaggle.com/datasets/shubhammaindola/harry-potter-books?resource=download  
alice - https://www.kaggle.com/datasets/leelatte/alicetxt

In [ ]:
"/mnt/d/Data/llm_data"

In [14]:
import re


def clean_data(filr_dir, file_name):

    with open(f"{filr_dir}/{file_name}", "r", encoding="utf-8") as file:
        book_text = file.read()

    cleaned_data = re.sub(r"\n+", " ", book_text)  # 줄 바꿈을 하나의 공백으로 변경
    cleaned_data = re.sub(r"\s", " ", cleaned_data)  # 여러 공백을 하나의 공백으로 변경

    print(f"cleaned_{file_name, len(cleaned_data)} characters")  # 글자 수 출력

    save_dir = filr_dir[:-9]
    # save_file_name = file_name.split("/")[-1]

    with open(f"{save_dir}/cleaned/{file_name}", "w", encoding="utf-8") as file:
        file.write(cleaned_data)

In [15]:
import os

harry_potter_dir = "/mnt/d/Data/llm_data/harry_potter/original"
harry_potter_file = os.listdir(harry_potter_dir)

for file in harry_potter_file:
    clean_data(harry_potter_dir, file)

cleaned_('01 Harry Potter and the Sorcerers Stone.txt', 436001) characters
cleaned_('02 Harry Potter and the Chamber of Secrets.txt', 488771) characters
cleaned_('03 Harry Potter and the Prisoner of Azkaban.txt', 621137) characters
cleaned_('04 Harry Potter and the Goblet of Fire.txt', 1093670) characters
cleaned_('05 Harry Potter and the Order of the Phoenix.txt', 1489734) characters
cleaned_('06 Harry Potter and the Half-Blood Prince.txt', 982041) characters
cleaned_('07 Harry Potter and the Deathly Hallows.txt', 1133063) characters


# Tokenizer

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

text = "Harry Potter was a wizard."

tokens = tokenizer.encode(text)

print(f"글자 수 : {len(text)}, 토큰 수 : {len(tokens)}")
print(tokens)  # 인코딩
print(tokenizer.decode(tokens))  # 디코딩

for t in tokens:
    print(f"{t}\t -> {tokenizer.decode([t])}")

글자 수 : 26, 토큰 수 : 6
[18308, 14179, 373, 257, 18731, 13]
Harry Potter was a wizard.
18308	 -> Harry
14179	 ->  Potter
373	 ->  was
257	 ->  a
18731	 ->  wizard
13	 -> .


In [ ]:
for char in text:
    token_ids = tokenizer.encode(char)  # 한 글자씩 인코딩
    decoded = tokenizer.decode(token_ids)  # 한 글자씩 디코딩
    print(f"{char} -> {token_ids} -> {decoded}")
    # 공백, 마침표도 토큰화 됨

H -> [39] -> H
a -> [64] -> a
r -> [81] -> r
r -> [81] -> r
y -> [88] -> y
  -> [220] ->  
P -> [47] -> P
o -> [78] -> o
t -> [83] -> t
t -> [83] -> t
e -> [68] -> e
r -> [81] -> r
  -> [220] ->  
w -> [86] -> w
a -> [64] -> a
s -> [82] -> s
  -> [220] ->  
a -> [64] -> a
  -> [220] ->  
w -> [86] -> w
i -> [72] -> i
z -> [89] -> z
a -> [64] -> a
r -> [81] -> r
d -> [67] -> d
. -> [13] -> .


# Dataloader

In [19]:
import torch
from torch.utils.data import Dataset, DataLoader


class LLM_Dataset(Dataset):
    def __init__(self, txt, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        print(f"# of tokens in txt: {len(token_ids)}")

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
with open("/mnt/d/Data/llm_data/harry_potter/cleaned/01 Harry Potter and the Sorcerers Stone.txt", "r", encoding="utf-8-sig") as file:
    txt = file.read()

dataset = LLM_Dataset(txt=txt, max_length=32, stride=4)

train_loader = DataLoader(dataset=dataset, batch_size=128, shuffle=True, drop_last=True)

# of tokens in txt: 117768


In [21]:
data_iter = iter(train_loader)

x, y = next(data_iter)

print(tokenizer.decode(x[0].tolist()))
print(tokenizer.decode(y[0].tolist()))

?” said the small voice. “Are you sure? You could be great, you know, it’s all here in your head,
” said the small voice. “Are you sure? You could be great, you know, it’s all here in your head, and


In [ ]:
cleaned_harry_potter_dir = "/mnt/d/Data/llm_data/harry_potter/cleaned"